# Question 8 : "THE LEAGUE" (16 points)

### Description

In this football problem, you will work like a data scientist and use real world data to perform a Bayesian Update on the distribution of the Goal Scoring Rate for two teams, Manchester United and Liverpool. 


### Questions 

a. Import the necessary libraries, read the "league_tables.csv" file, and remove rows that are not Liverpool or Manchester United. (2 points)

b. Create two separate dataframes for the two teams and calculate the average goal scoring rate (sum of GF / sum of MP) for each team. (3 points)

c. Fit the gamma distribution for both teams. (2 points)

d. (Given the Poisson likelihood function and update function from the textbook) : Find the updated distribution for Liverpool and Manchester United. For each, also graph the Prior and Posterior distributions in the same plot. (6 points)

e. Suppose Manchester and Liverpool play each other and the resulting score is 3 (Manchester) - 2 (Liverpool). How certain are we that Manchester is the better team? 


### Part a 

### Part b

### Part c

### Functions (from textbook)

In [9]:
def likelihood_poisson(lam, data):
    return [poisson.pmf(data, l)  for l in lam]

def update(distribution, likelihood):
    distribution['probs'] = distribution['probs'] * likelihood
    prob_data = distribution['probs'].sum()
    distribution['probs'] = distribution['probs'] / prob_data
    return distribution

### Part d

### Part e 